In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [2]:
spark = SparkSession.builder.appName('Linear Regression').getOrCreate()

23/12/24 00:03:39 WARN Utils: Your hostname, DESKTOP-0HJMRJF resolves to a loopback address: 127.0.1.1; using 172.25.116.190 instead (on interface eth0)
23/12/24 00:03:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/24 00:03:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/24 00:03:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/12/24 00:03:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
spark

In [4]:
training = spark.read.format("libsvm").load('data/ML/sample_linear_regression_data.txt')

23/12/24 00:03:45 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [6]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction', regParam=0.0)

In [7]:
lrModel = lr.fit(training)

23/12/24 00:03:51 WARN Instrumentation: [a8d20665] regParam is zero, which might cause numerical instability and overfitting.
23/12/24 00:03:51 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [8]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [9]:
lrModel.intercept

0.14228558260358093

In [10]:
training_summary = lrModel.summary

In [11]:
training_summary.r2

0.027839179518600154

In [12]:
all_data = spark.read.format('libsvm').load('data/ML/sample_linear_regression_data.txt')

23/12/24 00:03:51 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [13]:
train_data,test_data = all_data.randomSplit([0.7, 0.3])

In [14]:
train_data

DataFrame[label: double, features: vector]

In [15]:
test_data

DataFrame[label: double, features: vector]

In [16]:
correct_model = lr.fit(train_data)

23/12/24 00:05:21 WARN Instrumentation: [c9fab8ae] regParam is zero, which might cause numerical instability and overfitting.


In [18]:
test_result = correct_model.evaluate(test_data)

In [22]:
test_result.rootMeanSquaredError

9.930231565876422

In [24]:
unlabeled_data = test_data.select('features')

In [25]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+


In [26]:
predictions = correct_model.transform(unlabeled_data)

In [27]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|  1.1752833556560909|
|(10,[0,1,2,3,4,5,...| -1.2283016814247565|
|(10,[0,1,2,3,4,5,...| -0.5915038049336947|
|(10,[0,1,2,3,4,5,...|  1.4224301175233605|
|(10,[0,1,2,3,4,5,...|   1.218860002842078|
|(10,[0,1,2,3,4,5,...|  1.3077420876681076|
|(10,[0,1,2,3,4,5,...|  2.7014502613258853|
|(10,[0,1,2,3,4,5,...|  0.8764239582533748|
|(10,[0,1,2,3,4,5,...| -0.9890906272873339|
|(10,[0,1,2,3,4,5,...|  3.2476119854135064|
|(10,[0,1,2,3,4,5,...| -0.1936070221188782|
|(10,[0,1,2,3,4,5,...| -1.3883723547326277|
|(10,[0,1,2,3,4,5,...|-0.04808824670752371|
|(10,[0,1,2,3,4,5,...| -1.2202738492350695|
|(10,[0,1,2,3,4,5,...|  -2.323518136251651|
|(10,[0,1,2,3,4,5,...|  1.0489660950660633|
|(10,[0,1,2,3,4,5,...|   2.180335475863545|
|(10,[0,1,2,3,4,5,...|  -0.937098382140678|
|(10,[0,1,2,3,4,5,...|  0.9920902239171738|
|(10,[0,1,2,3,4,5,...|    1.4125